A more complete notebook can be found [here](https://colab.research.google.com/github/google-deepmind/mujoco/blob/main/python/tutorial.ipynb)

In [1]:
import mujoco
import mujoco_viewer

# scene.xml includes the robot model and a simple environment
model = mujoco.MjModel.from_xml_path("../unitree_go1/scene_torque.xml")

# Contains the state of the model (time: .time, pos: .qpos, vel: .qvel) 
data = mujoco.MjData(model)

In [2]:
# Print out the name of the body accessors
try:
    model.geom()
except KeyError as e:
    print(f"model.geom: {e}")
    
try:
    data.geom()
except KeyError as e:
    print(f"data.geom: {e}")
    
try:
    data.body()
except KeyError as e:
    print(f"data.body: {e}")

model.geom: "Invalid name ''. Valid names: ['FL', 'FR', 'RL', 'RR', 'floor']"
data.geom: "Invalid name ''. Valid names: ['FL', 'FR', 'RL', 'RR', 'floor']"
data.body: "Invalid name ''. Valid names: ['FL_calf', 'FL_hip', 'FL_thigh', 'FR_calf', 'FR_hip', 'FR_thigh', 'RL_calf', 'RL_hip', 'RL_thigh', 'RR_calf', 'RR_hip', 'RR_thigh', 'trunk', 'world']"


In [3]:
model.geom("FL")

<_MjModelGeomViews
  bodyid: array([7])
  conaffinity: array([1])
  condim: array([6])
  contype: array([1])
  dataid: array([-1])
  friction: array([0.8 , 0.02, 0.01])
  gap: array([0.])
  group: array([3])
  id: 31
  margin: array([0.001])
  matid: array([-1])
  name: 'FL'
  pos: array([ 0.   ,  0.   , -0.213])
  priority: array([1])
  quat: array([1., 0., 0., 0.])
  rbound: array([0.023])
  rgba: array([0.5, 0.5, 0.5, 1. ], dtype=float32)
  sameframe: array([0], dtype=uint8)
  size: array([0.023, 0.   , 0.   ])
  solimp: array([0.015, 1.   , 0.031, 0.5  , 2.   ])
  solmix: array([1.])
  solref: array([0.02, 1.  ])
  type: array([2])
  user: array([], dtype=float64)
>

In [4]:
print(f"Simulated time: {data.time} sec")
print(f"Default timestep: {model.opt.timestep} sec")
print(f"Degrees of freedom: {model.nv=}")

# Position has 1 extra dimension since orientation is represented with
# quaternions (4-values) while angular velocity has 3 values.
print(f"{len(data.qpos)=} {data.qpos=}")
print(f"{len(data.qvel)=} {data.qvel=}")

Simulated time: 0.0 sec
Default timestep: 0.002 sec
Degrees of freedom: model.nv=18
len(data.qpos)=19 data.qpos=array([0.   , 0.   , 0.445, 1.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   ])
len(data.qvel)=18 data.qvel=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.])


In [12]:
import numpy as np
import time
# Setup viewer
viewer = mujoco_viewer.MujocoViewer(model, data)

# Reset the simulation
mujoco.mj_resetDataKeyframe(model, data, 0)

# data.qpos[5] = np.deg2rad(15)
# print(f"{data.site_xpos=}")
print(f"{len(data.ctrl)=} {data.ctrl=}")
data.ctrl[0] = 1
for _ in range(1000):
    if viewer.is_alive:
        mujoco.mj_step(model, data)
        mujoco.mj_forward(model, data)
        mujoco.mj_rnePostConstraint(model, data)
        viewer.render()
        time.sleep(0.01)
        # 6 arrays, each wi
        # th 14 values
        # [4][10]: rightside, [7][13]: leftside
        # [7][4]: front
        # 4:fromright
        # 7:frontleft
        # 10:backright
        # 13:backleft
        # print(f"{len(data.cfrc_ext)=} {data.cfrc_ext[0][0].shape=} {data.cfrc_ext[4]=}")
        # sites in MJCF models: https://mujoco.readthedocs.io/en/stable/XMLreference.html#body-site
        # 6 in the Go1 model xml file
        feet_site = [
            'FR',
            'FL',
            'RR',
            'RL',
        ]
        feet_site_name_to_id = {
            f: mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_SITE.value, f)
            for f in feet_site
        }
        # print(f"{feet_site_id=}")
        # print(f"z heights {data.site_xpos[2:, 2]=}")
        # print(f"{data.site_xpos[feet_site_name_to_id['RR']]=}")
        # print(f"{data.xpos=}")
    else:
        break

viewer.close()

len(data.ctrl)=12 data.ctrl=array([ 0. ,  0.9, -1.8,  0. ,  0.9, -1.8,  0. ,  0.9, -1.8,  0. ,  0.9,
       -1.8])


AttributeError: 'mujoco._structs.MjData' object has no attribute 'ctrlrange'